In [0]:
df = spark.read.csv("/Volumes/workspace/default/e-commerce/ecommerce_capstone_dataset_1000.csv", 
                    header=True, inferSchema=True)

display(df)
df.printSchema()
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
segment_indexer = StringIndexer(inputCol="customer_segment", outputCol="segment_index")

assembler = VectorAssembler(
    inputCols=["age", "gender_index", "num_orders", "avg_order_value", "clv_estimate"],
    outputCol="features"
)

lr = LogisticRegression(featuresCol="features", labelCol="churn_flag")

pipeline = Pipeline(stages=[gender_indexer, segment_indexer, assembler, lr])
model = pipeline.fit(df)

predictions = model.transform(df)
display(predictions.select("customer_id", "probability", "prediction", "churn_flag"))
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

assembler = VectorAssembler(
    inputCols=["age", "num_orders", "avg_order_value", "clv_estimate"],
    outputCol="features"
)

df_cluster = assembler.transform(df)

kmeans = KMeans(k=3, seed=42)
kmeans_model = kmeans.fit(df_cluster)

clustered_df = kmeans_model.transform(df_cluster)

# FIXED DISPLAY LINE
display(clustered_df.select("customer_id", "age", "num_orders", "avg_order_value", "clv_estimate", "prediction"))
from pyspark.sql.functions import col, count, row_number
from pyspark.sql.window import Window

# ==========================================================
# 1) Load Dataset (Your Correct Volume Path)
# ==========================================================
df = spark.read.option("header", True).option("inferSchema", True) \
    .csv("/Volumes/workspace/default/e-commerce/ecommerce_capstone_dataset_1000.csv")

# ==========================================================
# 2) Compute Product Popularity (Most purchased products)
# ==========================================================
product_popularity = (
    df.groupBy("product_id")
      .agg(count("*").alias("purchase_count"))
      .orderBy(col("purchase_count").desc())
)

# ==========================================================
# 3) Find products each customer has already purchased
# ==========================================================
customer_products = df.select("customer_id", "product_id").distinct()

# ==========================================================
# 4) Recommend products the customer has NOT purchased yet
# ==========================================================
# Cross-join customers with top products and remove purchased ones
recommendations = (
    df.select("customer_id").distinct()
      .crossJoin(product_popularity)
      .join(customer_products, on=["customer_id", "product_id"], how="left_anti")
)

# ==========================================================
# 5) Assign rank (Top N Recommendations Per Customer)
# ==========================================================
window = Window.partitionBy("customer_id").orderBy(col("purchase_count").desc())

ranked_recommendations = (
    recommendations
        .withColumn("rank", row_number().over(window))
        .filter(col("rank") <= 5)   # Top 5 recommendations per customer
)

display(ranked_recommendations)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(
    inputCols=["age", "num_orders", "avg_order_value"],
    outputCol="features"
)

df_reg = assembler.transform(df)

lr = LinearRegression(featuresCol="features", labelCol="clv_estimate")
lr_model = lr.fit(df_reg)

print("R² Score:", lr_model.summary.r2)
display(lr_model.summary.residuals)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

assembler = VectorAssembler(
    inputCols=["age", "num_orders", "avg_order_value", "clv_estimate"],
    outputCol="features"
)

df_churn = assembler.transform(df)

rf = RandomForestClassifier(featuresCol="features", labelCol="churn_flag", numTrees=50)
rf_model = rf.fit(df_churn)

predictions = rf_model.transform(df_churn)
display(predictions.select("customer_id", "churn_flag", "prediction", "probability"))

customer_id,age,gender,product_id,category,price,quantity,purchase_date,transaction_amount,num_orders,avg_order_value,customer_segment,clv_estimate,churn_flag
1102,22,Male,5001,Books,74.88,1,2023-08-03,74.88,9,74.88,High Value,8629.69,0
1435,50,Male,5023,Books,347.48,4,2024-07-14,1389.92,6,347.48,Low Value,26174.36,0
1860,18,Male,5033,Cosmetics,4528.59,4,2024-04-16,18114.36,2,4528.59,High Value,100237.58,0
1270,35,Female,5026,Groceries,3255.21,4,2023-09-04,13020.84,9,3255.21,Low Value,330112.62,0
1106,49,Male,5040,Groceries,1401.55,1,2024-01-17,1401.55,6,1401.55,High Value,95698.11,0
1071,28,Male,5000,Cosmetics,927.1,2,2023-08-27,1854.2,3,927.1,Medium Value,31393.13,1
1700,38,Female,5007,Cosmetics,1506.62,3,2024-01-02,4519.86,6,1506.62,Medium Value,95254.08,1
1020,43,Female,5022,Cosmetics,618.24,3,2023-06-17,1854.72,7,618.24,Medium Value,64327.24,0
1614,42,Female,5037,Groceries,4710.59,3,2024-07-13,14131.77,1,4710.59,Low Value,58048.16,0
1121,39,Male,5005,Cosmetics,3931.82,4,2023-02-20,15727.28,1,3931.82,Medium Value,50207.23,1


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

root
 |-- customer_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- transaction_amount: double (nullable = true)
 |-- num_orders: integer (nullable = true)
 |-- avg_order_value: double (nullable = true)
 |-- customer_segment: string (nullable = true)
 |-- clv_estimate: double (nullable = true)
 |-- churn_flag: integer (nullable = true)



...

residuals
23500.19010922211
89081.60107468608
-164770.05333215685
23491.6244240246
51674.08941812768
107651.66583610751
39794.97143375249
74727.20080574983
-199141.85979669876
-128245.85193190168


Databricks visualization. Run in Databricks to view.

customer_id,churn_flag,prediction,probability
1102,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.6898459375111083"",""0.31015406248889166""]}"
1435,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7398386756155969"",""0.26016132438440315""]}"
1860,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7838951462450277"",""0.21610485375497235""]}"
1270,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.771632332275049"",""0.22836766772495096""]}"
1106,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7824283724507544"",""0.21757162754924572""]}"
1071,1,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7579066993523803"",""0.2420933006476197""]}"
1700,1,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7836764927515061"",""0.2163235072484938""]}"
1020,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7891470863707624"",""0.2108529136292376""]}"
1614,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.8050192759292126"",""0.1949807240707874""]}"
1121,1,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7724420856074571"",""0.22755791439254291""]}"


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.